In [ ]:
import h5py
import msmrd2
import numpy as np
import matplotlib.pyplot as plt
import msmrd2.tools as msmrdtls
import msmrd2.analysis as msmrdals
from msmrd2.markovModels import msmrdMarkovModel
import pyemma
import matplotlib.pyplot as plt
import pyemma.plots as mplt

In [ ]:
# Define useful parametersdtrajs
dt = 0.000001
timesteps = 4000000 # 2000000
stride = 25 #2500
maxentries = int(timesteps/stride)
#fnamebase = '../data/dimer/old/simDimer'
fnamebase = '../data/dimer/simDimer_t4.00E+06_s25_'
nfiles = 200

In [ ]:
# Load full trajectories
trajs = []
for i in range(nfiles):
    traj = msmrdals.loadTrajectory(fnamebase, i)
    trajs.append(traj)
    print("File ", i+1, " of ", nfiles, " done.", end="\r")

In [ ]:
# Plot relative position
diffpos =[]
file=2
for entry in range(maxentries):
    time = trajs[file][entry][0]*dt
    part1pos = np.array(trajs[file][2*entry][1:4])
    part2pos = np.array(trajs[file][2*entry+1][1:4])
    diffpos.append(np.linalg.norm(part1pos - part2pos))
plt.plot(diffpos)

In [ ]:
# Plot relative orientation
difforient = []
difforientquat = []
for entry in range(int(maxentries)):
    time = trajs[file][entry][0]*dt
    part1orient = np.array(trajs[file][2*entry][4:8])
    part2orient = np.array(trajs[file][2*entry+1][4:8])
    relorient = msmrdtls.quaternions.relativeOrientation(part1orient, part2orient)
    phi = msmrdtls.quaternions.quat2angle(relorient)
    quatdist = msmrdtls.quaternionDistance(np.array([1,0,0,0]), relorient)
    difforient.append(np.linalg.norm(phi))
    difforientquat.append(np.linalg.norm(quatdist))
plt.plot(difforientquat, alpha = 0.5) #blue
plt.plot(np.array(difforient)/(2*np.pi), alpha = 0.5) #orange
#plt.plot([0,80000], [0.5, 0.5], '--')
#plt.plot([0,80000], [0.7, 0.7], '--')
#plt.plot([0,80000], [0.3, 0.3], '--')

In [ ]:
# Load discrete trajectories
dtrajs = []
for i in range(nfiles):
    dtraj = msmrdals.loadDiscreteTrajectory(fnamebase, i)
    dtrajs.append(dtraj)    
    print("File ", i+1, " of ", nfiles, " done.", end="\r")

In [ ]:
rateDict = msmrdals.extractRatesMSM(dtrajs, 200, 2)

In [ ]:
rateDict

In [ ]:
np.where(dtrajs[3]<0)

In [ ]:
slicedDtrajs = msmrdals.splitDiscreteTrajs(dtrajs,0)

In [ ]:
stitchedTrajs = msmrdals.stitchTrajs(slicedDtrajs, 500)

In [ ]:
lenlist = []
for traj in stitchedTrajs: #slicedDtrajs:
    lenlist.append(len(traj))

In [ ]:
plt.hist(np.log10(lenlist), bins=100);

In [ ]:
maxlagtime = 500
lagtime = 50

In [ ]:
myMSMs = []
numMSMs = 10
for i in range(numMSMs):
    mymsm = pyemma.msm.estimate_markov_model(stitchedTrajs,(i+1)*lagtime,reversible=True)
    myMSMs.append(mymsm)

mymsm = myMSMs[0]
#mymsm = pyemma.msm.estimate_markov_model(stitchedTrajs,lagtime,reversible=True)

In [ ]:
activeset = mymsm.active_set
# np.where(activeset == 66)
activeset

In [ ]:
its = pyemma.msm.its(stitchedTrajs, lags = maxlagtime)
mplt.plot_implied_timescales(its, nits = 40, ylog=True, units='steps', linewidth=2, dt=1)
plt.ylabel(r"timescale/steps", fontsize = 24)
plt.xlabel(r"lag time/steps", fontsize = 24)

In [ ]:
its.timescales[:,1]

In [ ]:
np.linalg.norm(mymsm.eigenvectors_right()[1])

In [ ]:
for i in [8]:#range(10):
    normi = np.linalg.norm(mymsm.eigenvectors_left()[i])
    plt.plot(mymsm.eigenvectors_left()[i]/normi);
#plt.xlim([0,10])

In [ ]:
myMSMs[3].mfpt(10,1)

In [ ]:
for state in range(212):
    s1 = state
    s2 = 1
    #print(mymsm.mfpt(s1,s2), mymsm2.mfpt(s1,s2), mymsm3.mfpt(s1,s2), mymsm4.mfpt(s1,s2))
    if (s1 != s2):
        try:
            mfpts = []
            for i in range(numMSMs):
                mfpts.append(myMSMs[i].mfpt(s1,s2))
            plt.plot(np.log(mfpts))
        except: 
            continue

            
plt.xlim([0,9])
#plt.ylim([9,10])
#plt.show()

In [ ]:
mytpt = pyemma.msm.tpt(mymsm,[s1],[s2])
mytpt2 = pyemma.msm.tpt(mymsm2,[s1],[s2])
mytpt3 = pyemma.msm.tpt(mymsm3,[s1],[s2])
mytpt4 = pyemma.msm.tpt(mymsm4,[s1],[s2])

print(mytpt.mfpt, mytpt2.mfpt, mytpt3.mfpt, mytpt4.mfpt)

In [ ]:
from msmtools.analysis import mfpt

In [ ]:
print(mfpt(mymsm.transition_matrix, origin = 4, target = 0, tau = lagtime),
      mfpt(mymsm2.transition_matrix, origin = 4, target = 0, tau = lagtime*2),
      mfpt(mymsm4.transition_matrix, origin = 4, target = 0, tau = lagtime*4),
      mfpt(mymsm4.transition_matrix, origin = 4, target = 0, tau = lagtime*8))

In [ ]:
origin = 1
target = 0
print(-np.log(1.0 - mymsm.transition_matrix[origin][target])/lagtime,
      -np.log(1.0 - mymsm2.transition_matrix[origin][target])/(2*lagtime),
      -np.log(1.0 - mymsm4.transition_matrix[origin][target])/(4*lagtime),
      -np.log(1.0 - mymsm8.transition_matrix[origin][target])/(8*lagtime))

In [ ]:
mytpt = pyemma.msm.tpt(mymsm,[0],[1])
mytpt.mfpt

In [ ]:
1/(dt*mymsm.mfpt(0,1)*200)

In [ ]:
mytpt = pyemma.msm.tpt(mymsm,[15],[10])
mytpt.rate/dt

In [ ]:
# Create distionaries to keep track of specific events. Needed to calculate rates
boundstates = 2
orientations = 7
dt = trajs[0][2][0] - trajs[0][0][0]
rateDict, timeDict, eventDict = msmrdals.extractRates(dtrajs, boundstates, orientations)

In [ ]:
rateDict

In [ ]:
a=msmrdMarkovModel(3,6,-1,rateDict)

In [ ]:
a.getRate('b2->b1')

In [ ]:
msmA = a.getMSM('15a')

In [ ]:
msmA.getTransitionMatrix()

In [ ]:
part1.state